In [1]:
import pandas as pd 
import numpy as np
from numpy import array
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from ipynb.fs.full.Data_Processing import *
from sklearn import preprocessing
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
torch.set_printoptions(linewidth=120) #Display options for output
torch.set_grad_enabled(True) # Already on by default
torch.manual_seed(0)
from torch_lr_finder import LRFinder
import pickle
import torch.utils.data as data_utils
from collections import namedtuple

### CNN

In [2]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*4, 5)
        
    def forward(self, x):
        # Layer 1
        x = x.float()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.1)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.1)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.1)
        x = self.pooling3(x)

        # FC Layer
        x = x.view(-1, 4*2*4)
        x = self.fc1(x)
        return x
    
    # for 60 timepoints = 4* 10 * 8 or 4*24
    # 120 timepoints = 4* 2* 7 and -1
    # https://discuss.pytorch.org/t/runtimeerror-shape-1-400-is-invalid-for-input-of-size/33354
    # https://discuss.pytorch.org/t/valueerror-expected-input-batch-size-324-to-match-target-batch-size-4/24498/2

### CNN + RNN

https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*4, 5)
        
    def forward(self, x):
        # Layer 1
        x = x.float()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.1)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.1)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.1)
        x = self.pooling3(x)

        # FC Layer
        x = x.view(-1, 4*2*4)
#         x = self.fc1(x)
        return x
    

class Combine(nn.Module):
    def __init__(self):
        super(Combine, self).__init__()
        self.cnn = CNN()
        self.rnn = nn.LSTM(
            input_size=32, 
            hidden_size=16, 
            num_layers=1,
            batch_first=True)
        self.linear = nn.Linear(16 ,5)

    def forward(self, x):
        batch_size, C, timepoints, channels = x.size()
        c_in = x
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, -1 , c_out.shape[1])
        r_out, (h_n, h_c) = self.rnn(r_in)
        r_out2 = self.linear(r_out[:, -1, :])
        
        return F.log_softmax(r_out2, dim=1)
# X.shape - (#batch size, 1, #timepoints, #channels)

**Evaluate function returns values of different criteria like accuracy, precision etc.**
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [4]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted , multi_class="ovr"))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted), average='macro'))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted) , average='macro'))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted) , average='macro')
            recall = recall_score(Y, np.round(predicted) , average='macro')
            results.append(2*precision*recall/ (precision+recall))
    return results

**Generate random data**
    
*Data format:*

Datatype - float32 (both X and Y)

X.shape - (#samples, 1, #timepoints, #channels)

Y.shape - (#samples)

In [5]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()### Scale the data

In [6]:
def learning_rate_finder():
    model = EEGNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-7, weight_decay=1e-2)
    lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
    lr_finder.range_test(train_loader, end_lr=100, num_iter=100, step_mode="exp")
    lr_finder.plot()

In [7]:
def get_accuracy(train_loader, test_loader, dataset_type, net):
    correct = 0
    total = 0
    with torch.no_grad():
        if dataset_type == "test":
            loader = test_loader
        elif dataset_type == "train":
            loader = train_loader
        for data in loader:
            inputs = data[0].cuda(0)
            labels = data[1].cuda(0)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = (correct/total * 100)
    return acc

In [8]:
def train_network(X_train, train_loader,  model_type , verbose=False,):
    
    if model_type == 'CNN':
        net = EEGNet().cuda(0)
    else: 
        net = Combine().cuda(0)
    
    optimizer = optim.Adam(net.parameters(), lr = 0.001)

    preds_list = []
    labels_list = []
    for epoch in range(500):
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
        #Get Batch

            inputs = batch[0].cuda(0)
            labels = batch[1].cuda(0)


            preds = net(inputs) #Pass batch
        #     criterion=nn.BCEWithLogitsLoss()
            loss = F.cross_entropy(preds, labels.long()) #calculate loss

            optimizer.zero_grad()
            loss.backward()  #calculate gradients
            optimizer.step() #update weights


            preds_list.append(preds)
            labels_list.append(labels)
            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)
            
        if verbose == True:             
    #     Validation accuracy
    #     params = ["acc", "fmeasure"]
    #     print (params)
    #     print ("Train - ", evaluate(net, X_train, y_train_new, params))
            print("epoch: {0}, Accuracy {1}, loss: {2}".format(epoch, total_correct/len(X_train), loss))
        
    return net

In [9]:
from ipynb.fs.full.evaluation import *
data = load_file("examples/User_1_sampled_annotated_EEG.pickle")
data['inputs'][0].shape

(60, 8)

### Training and testing

In [10]:
sampled_file = "examples/User_1_sampled_annotated_EEG.pickle"
with open(sampled_file, 'rb') as handle:
    dt = pickle.load(handle)

labels = ["attention", "interest", "effort"]

results = []
for label in labels:
    
    X = dt["inputs"]
    y = dt[label]

    #Convert the categories into labels 
    le = LabelEncoder()
    y =  le.fit_transform(y)

    #Train/test split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

    # scale the data 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

    #Convert to 4D 
    X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1],X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0],1, X_test.shape[1],X_test.shape[2])

    #Create train and test loader
    train = data_utils.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
    test = data_utils.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
    train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)
    test_loader = data_utils.DataLoader(test, batch_size=50, shuffle=False)
    
    # train the network
    net = train_network(X_train, train_loader, 'hybrid', verbose=True)
    
    #get results
    Results = namedtuple("Results","label train_acc test_acc user")
    train_acc = get_accuracy(train_loader, test_loader, "train", net)
    test_acc = get_accuracy(train_loader, test_loader, "test", net)
    print("{0} results on User 1\ntrain_acc: {1}\tsample_size: {2}\ntest_acc: {3}\tsample_size: {4}\n".format(label, 
                                                                                           train_acc,  len(X_train),
                                                                                            test_acc, len(X_test)))
    results.append(Results(label, train_acc, test_acc,1))

results = pd.DataFrame(results)
results.to_csv("results/EEGNet/user1_results_sampled_annotated.csv", index=False)
results

epoch: 0, Accuracy 0.35094001790510293, loss: 1.4480037689208984
epoch: 1, Accuracy 0.36705461056401073, loss: 1.5189579725265503
epoch: 2, Accuracy 0.36615935541629363, loss: 1.4226410388946533
epoch: 3, Accuracy 0.37242614145031333, loss: 1.40126633644104
epoch: 4, Accuracy 0.40555058191584603, loss: 1.3762263059616089
epoch: 5, Accuracy 0.4297224709042077, loss: 1.1519502401351929
epoch: 6, Accuracy 0.4744852282900627, loss: 1.1013457775115967
epoch: 7, Accuracy 0.4726947179946285, loss: 1.3084444999694824
epoch: 8, Accuracy 0.4870188003581021, loss: 1.190678596496582
epoch: 9, Accuracy 0.5058191584601611, loss: 1.1995295286178589
epoch: 10, Accuracy 0.4986571172784244, loss: 1.1557328701019287
epoch: 11, Accuracy 0.4932855863921218, loss: 0.9991579651832581
epoch: 12, Accuracy 0.5040286481647269, loss: 1.0435608625411987
epoch: 13, Accuracy 0.5129811996418979, loss: 1.1422083377838135
epoch: 14, Accuracy 0.5058191584601611, loss: 1.0455299615859985
epoch: 15, Accuracy 0.51029543419

,label,train_acc,test_acc,user
0,attention,61.145927,67.500000,1
1,interest,67.949866,65.000000,1
2,effort,69.113697,63.214286,1


**Run**